# Parameters and Parameter Space

This notebook demonstrates usage of the Parameter and ParameterSpace classes in python. These classes can work standalone within python, as shown here, but they can also connect to a TINC server through the TincClient class.

In [1]:
# Not needed if tinc-python is installed
import sys
sys.path.append('../tinc-python')

from tinc_object import *
TincVersion()

0.9


## Parameters

In [2]:
from parameter import *

Parameters must have a unique name passed to the constructor

In [3]:
p = Parameter("temperature")

Parameters have a current value, that you can read and write to using the 'value' member

In [4]:
p.value

0.0

In [5]:
p.value = 30

In [6]:
p.value

30.0

### Boundaries

You can set the boundaries of the parameter using 'minimum'and 'maximum

In [7]:
p.minimum = 10
p.maximum = 30

In [8]:
p.value = 0
p.value

10.0

In [9]:
p.value = 32
p.value

30.0

## Defining possible values for parameter/dimension

You can set the possible values a parameter can take:

In [10]:
import numpy as np

In [11]:
p.values = np.linspace(10, 40, 10)

In [12]:
p.values

array([10.        , 13.33333333, 16.66666667, 20.        , 23.33333333,
       26.66666667, 30.        , 33.33333333, 36.66666667, 40.        ])

In [13]:
p.value = 50

Incoming values are clamped to the boundaies set by the valid values and rounded to the nearest possible value

In [14]:
p.value

40.0

In [15]:
p.value = 31
p.value

30.0

In [16]:
p.value = 32
p.value

33.333333333333336

### Value callbacks

You can register a callback function that can be triggered whenever a parameter's value changes. The callback function gets passed the new parameter value

In [17]:
def computation(value):
    print(f'Got {value}')

p.register_callback(computation)

In [18]:
p.value = 70

Got 40.0


### Value ids

Each possible value the parameter can take can have an associated id. This can be useful for example to map parameter to filesystem names.

In [19]:
chem_pot = Parameter("chemical_potential")
chem_pot.set_values([-0.3, -0.15, 0.0, 0.15, 0.3, 0.45])
chem_pot.set_ids(['-0.30', '-0.15', '0', '0.15', '0.30', '0.45'])

In [20]:
chem_pot.value = -0.15
chem_pot.get_current_id()

'-0.15'

In [21]:
chem_pot.value = -0.14
chem_pot.get_current_id()

'-0.15'

## Parameter Spaces

A ParameterSpace in TINC is a class that groups and manages parameters. In that sense, each parameter is a dimension of the parameter space.

In [22]:
from parameter_space import *

In [23]:
ps = ParameterSpace("ps1")

In [24]:
ps.register_parameters([p,chem_pot])

A parameter space can be associated with a filesystem path that can change according to its parameters

In [25]:
ps.set_current_path_template("t_%%temperature%%_chempot_%%chemical_potential:ID%%")

In [26]:
ps.get_current_path()

't_40.0_chempot_-0.15'

In [27]:
chem_pot.value = 0.45
ps.get_current_path()

't_40.0_chempot_0.45'

You can execute a function through the parameter space to have it be run with the parameter space's current values:

In [28]:
def processor(temperature, chemical_potential):
    print(f"Running at: T {temperature} -- chem_pot {chemical_potential}")

ps.run_process(processor)

Running at: T 40.0 -- chem_pot 0.45


In [29]:
p.value = 30
ps.run_process(processor)

Got 30.0
Running at: T 30.0 -- chem_pot 0.45


You can have the parameter space cache the output for these processes automatically, by enabling caching. You can determine where this cache gets stored by passing the foler name.

In [30]:
ps.enable_caching("cache")

In [31]:
import time

def long_processor(temperature, chemical_potential):
    print(f"Running at: T {temperature} -- chem_pot {chemical_potential}")
    time.sleep(3) # Simulate a time consuming process
    return temperature * chemical_potential

In [32]:
%%time
value = ps.run_process(long_processor)

Running at: T 30.0 -- chem_pot 0.45
Wall time: 3 s


In [33]:
value

13.5

In [34]:
%%time
value = ps.run_process(long_processor)
value

Using cache
Wall time: 1 ms


13.5

In [35]:
p.value = 10

Got 10.0


In [36]:
%%time
value = ps.run_process(long_processor)
value

Running at: T 10.0 -- chem_pot 0.45
Wall time: 3 s


4.5

In [37]:
%%time
value = ps.run_process(long_processor)
value

Using cache
Wall time: 1 ms


4.5

In [39]:
p.value = 30

Got 30.0


In [40]:
%%time
value = ps.run_process(long_processor)
value

Using cache
Wall time: 997 µs


13.5

### Parameter sweeps

You can run parameter sweeps generating cache:

In [41]:
%%time
ps.sweep(long_processor)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('Running at: T ')
              4 LOAD_FAST                0 (temperature)
              6 FORMAT_VALUE             0
              8 LOAD_CONST               2 (' -- chem_pot ')
             10 LOAD_FAST                1 (chemical_potential)
             12 FORMAT_VALUE             0
             14 BUILD_STRING             4
             16 CALL_FUNCTION            1
             18 POP_TOP

  5          20 LOAD_GLOBAL              1 (time)
             22 LOAD_METHOD              2 (sleep)
             24 LOAD_CONST               3 (3)
             26 CALL_METHOD              1
             28 POP_TOP

  6          30 LOAD_FAST                0 (temperature)
             32 LOAD_FAST                1 (chemical_potential)
             34 BINARY_MULTIPLY
             36 RETURN_VALUE
None
def long_processor(temperature, chemical_potential):
    print(f"Running at: T {temperature} -- chem_pot 